# Example: Comparison of Bielliptic and Hohmann Transfers

As we discussed, bielliptic transfers can save $\Delta v$ requirement when the initial and target orbits have very different radii. When the target orbit radius is more than about 15.5 times larger than the initial radius (or vice versa), the bielliptic transfer is more energy efficient than the standard Hohmann transfer.

In this example, we will explore this with some numbers, and also discuss the tradeoff of a bielliptic transfer.

Let's start the initial orbit in LEO orbit of 500 km altitude and transfer to an orbit at the radius of the moon, at 385,000 km. Assuming both orbits are circular, we can find the initial and target orbital velocities:

$$v = \sqrt{\frac{\mu}{r}}$$

In [1]:
import numpy as np

mu = 398_600.4418  # km**3/s**2
R_E = 6378  # km
r_1 = 500 + R_E  # km

v_1 = np.sqrt(mu / r_1)

r_3 = 385_000
v_3 = np.sqrt(mu / r_3)

print(f"{v_1=:.3F} km/s", f"{v_3=:.3F} km/s")

v_1=7.613 km/s v_3=1.018 km/s


Now we will calculate the standard two-impulse Hohmann transfer. The total $\Delta v$ for the transfer is the sum of the $\Delta v$ from the initial orbit onto the transfer orbit at perigee of the transfer, and the $\Delta v$ from the transfer orbit onto the target orbit at the apogee of the transfer.

First, we calculate the eccentricity, specific orbital angular momentum, and semimajor axis of the transfer orbit:

$$e = \frac{r_3 - r_1}{r_3 + r_1}$$

$$h = \sqrt{\mu r_3 (1 - e)} = \sqrt{\mu r_1 (1 + e)}$$

$$a = \frac{r_3 + r_1}{2}$$

In [2]:
e_h = (r_3 - r_1) / (r_1 + r_3)
h_h = np.sqrt(r_3 * (1 - e_h) * mu)
a_h = (r_3 + r_1) / 2
print(f"{e_h=:.4F}", f"{h_h=:.2F} km**2/s", f"{a_h=:.0F} km")

e_h=0.9649 h_h=73395.58 km**2/s a_h=195939 km


Now, the velocity at the perigee and apogee of the transfer orbit are determined from the specific orbital angular momentum.

In [3]:
v_p_h = h_h / r_1
v_a_h = h_h / r_3
print(f"{v_p_h=:.3F} km/s", f"{v_a_h=:.3F} km/s")

v_p_h=10.671 km/s v_a_h=0.191 km/s


Notice that the velocity at the apogee of the transfer orbit is only about 190 m/s, but the orbital velocity at the moon's orbit is over 1,000 m/s. Now, let's calculate $\Delta v$:

$$\Delta v_h = v_{p,h} - v_1 + v_3 - v_{a,h}$$

In [4]:
delta_v_h = v_p_h - v_1 + v_3 - v_a_h
print(f"{delta_v_h=:.3F} km/s")

delta_v_h=3.885 km/s


So $\Delta v$ for the Hohmann transfer is about 3.885 km/s. Next, let's do the bielliptic transfer. We (somewhat arbitrarily) pick $r_2 =$ 800,000 km, that is, apogee of the transfer orbit is 800,000 km from the center of the Earth. Now, we can calculate the eccentricity, specific orbital angular momentum, and velocities for the two transfer ellipses.

In [5]:
r_2 = 800_000
e_1 = (r_2 - r_1) / (r_1 + r_2)
h_1 = np.sqrt(r_2 * (1 - e_1) * mu)
v_p_1 = h_1 / r_1
v_a_1 = h_1 / r_2

e_2 = (r_2 - r_3) / (r_3 + r_2)
h_2 = np.sqrt(r_2 * (1 - e_2) * mu)
v_p_2 = h_2 / r_3
v_a_2 = h_2 / r_2

print(f"{v_a_1=:.4F} km/s", f"{v_a_2=:.4F} km/s", f"{v_p_2=:.4F} km/s")

v_a_1=0.0922 km/s v_a_2=0.5690 km/s v_p_2=1.1823 km/s


The difference in apogee velocities between the two transfer orbits is about 0.4 km/s, and now the insertion velocity to the lunar radius orbit ($v_{p,2}$) is about 1.18 km/s, much closer to the lunar orbit speed. The total $\Delta v$ for these three impulses is:

In [6]:
delta_v_2 = v_p_1 - v_1 + v_a_2 - v_a_1 + v_p_2 - v_3
print(f"{delta_v_2=:.4F} km/s")

delta_v_2=3.7489 km/s


For the bielliptic transfer, $\Delta v$ is about 3.75 km/s, about 130 m/s less than the straight Hohmann transfer. Assuming a 1000 kg spacecraft, this results in a savings of propellant of:

In [7]:
delta_m_h = 1000 * (1 - np.exp(-delta_v_h / (300 * 9.81E-3)))
delta_m_be = 1000 * (1 - np.exp(- delta_v_2 / (300 * 9.81E-3)))
print(f"{delta_m_h=:.0F} kg", f"{delta_m_be=:.0F} kg")

delta_m_h=733 kg delta_m_be=720 kg


The difference is 13 kg per 1000 kg of spacecraft mass. For the Falcon 9, the [Full Thrust](https://en.wikipedia.org/wiki/Falcon_9_Full_Thrust) variant has a mass of 549,000 kg. The savings from the bielliptic transfer means that about 7,000 kg of fuel can be diverted to another use. The total payload capacity to Low Earth Orbit is about [23,000 kg](https://www.spacex.com/vehicles/falcon-9/), so this is a significant savings. Although this is a simplistic model of the rocket, we can at least see the order of magnitude of savings that are possible.

On the other hand, the downside of the bielliptic transfer is the transit time. The period of an ellipse is:

$$T = \frac{2\pi}{\sqrt{\mu}}a^{3/2}$$

In [8]:
a_h = (r_1 + r_3) / 2
t_h = np.pi / np.sqrt(mu) * a_h**(3/2)
a_be_1 = (r_2 + r_1) / 2
a_be_2 = (r_2 + r_3) / 2
t_be = np.pi / np.sqrt(mu) * (a_be_1**(3/2) + a_be_2**(3/2))
from datetime import timedelta
print(timedelta(seconds=t_h), timedelta(seconds=t_be))

4 days, 23:53:00.998726 41 days, 0:35:23.331550


The transfer time is over 8x longer for the bielliptic transfer, going from 5 days to almost 1.5 months.